In [ ]:
x = -15132376222941642752 
k = 12
r = (x**4-x**2+1)
q = ((x-1)**2)//3 * r + x
h = (q**k - 1) // r

lam = 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129030796414117214202539
m = lam // r

p = 5044125407647214251
h3 = 2366356426548243601069753987687709088104621721678962410379583120840019275952471579477684846670499039076873213559162845121989217658133790336552276567078487633052653005423051750848782286407340332979263075575489766963251914185767058009683318020965829271737924625612375201545022326908440428522712877494557944965298566001441468676802477524234094954960009227631543471415676620753242466901942121887152806837594306028649150255258504417829961387165043999299071444887652375514277477719817175923289019181393803729926249507024121957184340179467502106891835144220611408665090353102353194448552304429530104218473070114105759487413726485729058069746063140422361472585604626055492939586602274983146215294625774144156395553405525711143696689756441298365274341189385646499074862712688473936093315628166094221735056483459332831845007196600723053356837526749543765815988577005929923802636375670820616189737737304893769679803809426304143627363860243558537831172903494450556755190448279875942974830469855835666815454271389438587399739607656399812689280234103023464545891697941661992848552456326290792224091557256350095392859243101357349751064730561345062266850238821755009430903520645523345000326783803935359711318798844368754833295302563158150573540616830138810935344206231367357992991289265295323280

assert(h == 27 * p * h3)
assert(m == 3 * p^2)

assert(gcd(3, h3) == 1)
assert(gcd(p^2, h3) == 1)
assert(gcd(3, h3) == 1)
assert(gcd(p, h3) == 1)
assert(gcd(p, 27 * h3) == 1)
assert(gcd(27, p * h3) == 1)

F = GF(q)
Fq12.<u> = GF(q^12)
ONE = Fq12(1)

w27 = ONE.nth_root(27); 
wp = ONE.nth_root(p);

assert(w27^9 != ONE)

In [ ]:
from random import randrange
def sample_27th_root(): 
    i = randrange(27)
    return [w27^i, i]

def sample_pth_root(): 
    i = randrange(p)
    return [wp^i, i]

def sample_h3_root():
    x = Fq12.random_element(); 
    x = x^r 
    x = x^27 
    x = x^p 
    
    assert(x^h3 == ONE)
    
    return x

In [ ]:
def has_27th_root_contribution(x): 
    # checks if x has a 27th root of unity contribution 
    try: 
        x.nth_root(3)
        return False
    except: 
        return True
     
def find_27th_root(x): 
    assert(has_27th_root_contribution(x))
    
    v = p * h3 
    v_inv = inverse_mod(v, 27)
    
    wj = x^v 
    j = discrete_log(wj, w27, 27)
    
    i = (j * v_inv) % 27
    return i

def clear_27th_root(x): 
    if not has_27th_root_contribution(x): 
        return [ONE, x] 
    
    i = find_27th_root(x)
    w_inv = (w27^i).inverse() 
    return [w_inv, x * w_inv] 
    
def has_pth_root_contribution(x): 
    # checks if x has a pth root of unity contribution 
    try: 
        x.nth_root(p)
        return False
    except: 
        return True
    
def find_pth_root(x): 
    assert(has_pth_root_contribution(x))
    
    v = 27 * h3 
    v_inv = inverse_mod(v, p)
    
    wj = x^v 
    j = discrete_log(wj, wp, p)
    
    i = (j * v_inv) % p
    return i

def clear_pth_root(x): 
    if not has_pth_root_contribution(x): 
        return [ONE, x] 
    
    i = find_pth_root(x)
    w_inv = (wp^i).inverse() 
    return [w_inv, x * w_inv]

In [ ]:
# now mock the pairing 
for i in range(100): 
    f = Fq12.random_element()
    f = f^r # it will always come as r-th residue in the honest execution

    [wp_shift, f_c1] = clear_pth_root(f)
    [w27_shift, f_c2] = clear_27th_root(f_c1)

    root = f_c2.nth_root(lam)
    assert(f * wp_shift * w27_shift, root^lam)
    print('step: ', i)